In [ ]:
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip install layoutparser

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

%cd drive/MyDrive/DLA

In [ ]:
import cv2
import layoutparser as lp

model = lp.Detectron2LayoutModel('lp://TableBank/faster_rcnn_R_101_FPN_3x/config',
                                 './outputs/model_final_table.pth',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.3],
                                 label_map={0: "Table"})


image = cv2.imread('./data/test_image.jpg')

color_map = {
    'Table':  'yellow',
}

layout_predicted = model.detect(image)
lp.draw_box(image,
              [b.set(id=f'{b.type}/{b.score:.2f}') for b in layout_predicted],
              color_map=color_map,
              show_element_id=True, id_font_size=20,
              id_text_background_color='black',
              id_text_color='white')

In [ ]:
from google.colab.patches import cv2_imshow
image = cv2.imread('./data/test_image.jpg')

margin = 20  # This is the margin size you want to add
for table in layout_predicted:
    # Adjust the coordinates to add a margin
    x1, y1, x2, y2 = table.coordinates
    table_crop = image[max(int(y1)-margin, 0):min(int(y2)+margin, image.shape[0]),
                       max(int(x1)-(margin+20), 0):min(int(x2)+(margin+20), image.shape[1])]

    gray = cv2.cvtColor(table_crop, cv2.COLOR_BGR2GRAY)


    # 가우시안 블러 적용 (노이즈 제거)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)



    # 어댑티브 이진화 적용
    adaptive_thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                            cv2.THRESH_BINARY_INV, 11, 4)


    # Canny 에지 검출
    edges = cv2.Canny(adaptive_thresh, threshold1=20, threshold2=150)

    # 에지를 사용하여 윤곽 찾기
    contours, _ = cv2.findContours(edges, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # 윤곽을 이미지에 그리기
    for cnt in contours:
        # 윤곽을 둘러싼 최소 사각형을 계산
        x, y, w, h = cv2.boundingRect(cnt)
        # 적절한 크기의 윤곽을 선택
        if w > 0 and h > 12:  # 셀 크기에 맞게 조정
            cv2.rectangle(table_crop, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2_imshow(table_crop)

